<a href="https://colab.research.google.com/github/DSabarish/netflix_data_analysis/blob/main/Netflix_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
'''Sabarish S'''

'Sabarish S'

# **Libraries and Data Quality Stats Function**

In [54]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Setting pandas options to display all columns and rows when printing dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# URL of the dataset to be read into a pandas dataframe
url = "https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/000/940/original/netflix.csv"
df = pd.read_csv(url)


In [55]:
def data_quality_statistics(df):
    """
    This function takes a DataFrame as input and returns a DataFrame containing data quality statistics
    for each column in the DataFrame.

    Args:
      df: A Pandas DataFrame.

    Returns:
      A DataFrame containing data quality statistics for each column.
    """

    num_rows = len(df)
    stats = []

    # Calculating statistics of Each column
    for col in df.columns:
        num_unique = df[col].nunique()
        num_nan = df[col].isnull().sum()
        pct_nan = (num_nan / num_rows) * 100
        data_type = df[col].dtype

        # Append statistics to the list
        stats.append({
            'Column': col,
            'Number_of_Rows': num_rows,
            'Number_of Unique Values': num_unique,
            'Number_of_NaN': num_nan,
            'Percentage_of_NaN': round(pct_nan, 2),
            'Data_Type': data_type,
        })

    stats_df = pd.DataFrame(stats)
    return stats_df

In [56]:
data_quality_statistics(df)

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
0,show_id,8807,8807,0,0.00,object
1,type,8807,2,0,0.00,object
2,title,8807,8807,0,0.00,object
3,director,8807,4528,2634,29.91,object
4,cast,8807,7692,825,9.37,object
5,country,8807,748,831,9.44,object
6,date_added,8807,1767,10,0.11,object
7,release_year,8807,74,0,0.00,int64
8,rating,8807,17,4,0.05,object
9,duration,8807,220,3,0.03,object


# **Mode Imputation Function**

> **Filtering out null directors, grouping by category, country, and type. <br>Sorting directors by frequency, capturing the first. Displaying grouped data.**

1. **Missing Value Elimination**: Initially, the function removes rows from the DataFrame (`df`) where the target feature (`target_feature`) has null values, ensuring a clean dataset for further processing.

2. **Grouping by Helper Features**: The function then groups the cleaned DataFrame based on auxiliary features (`mode_helper_features`). This grouping operation segments the data for subsequent analysis.

3. **Determining Most Frequent Value**: Within each group, the function calculates the most frequent occurrence of the target feature. This involves sorting the values by frequency and selecting the most common value.

4. **Filling Missing Values**: After determining the most frequent value for each group, the function proceeds to fill in missing values in the target feature column. It iterates through each row of the original DataFrame and replaces null values with the most common value from the corresponding group.

5. **Ensuring Data Integrity**: By replacing missing values with the most frequent values from their respective groups, the function ensures that the imputed data maintains integrity and reflects the overall distribution of the dataset.

In [57]:
import pandas as pd

def mode_based_imputation(df, target_feature, mode_helper_features):
    # Drop rows where target_feature is null
    filtered_df = df.dropna(subset=[target_feature])

    # Define a function to sort values by frequency
    def sort_values_by_frequency(values):
        if values.empty:
            return None, None
        value_count = values.value_counts()  # Count occurrences of each value
        sorted_values = value_count.index.tolist()  # Sort values by frequency
        return ', '.join(sorted_values), sorted_values[0]

    # Group by and perform string aggregation with sorted values
    grouped_df = (filtered_df
                  .groupby(mode_helper_features)[target_feature]
                  .apply(sort_values_by_frequency)
                  .apply(pd.Series)
                  )

    # Rename columns
    grouped_df.columns = ['sorted_values', 'first_value']

    # Reset index
    grouped_df = grouped_df.reset_index()

    # Create a dictionary mapping group keys to first values
    group_first_values = {}
    for index, row in grouped_df.iterrows():
        key = tuple(row[mode_helper_features])
        group_first_values[key] = row["first_value"]

    # Fill missing values in df[target_feature] based on the mapping
    for index, row in df.iterrows():
        key = tuple(row[mode_helper_features])
        if pd.isna(row[target_feature]):
            df.at[index, target_feature] = group_first_values.get(key, row[target_feature])

    return df

# Example usage:
# imputed_df = mode_based_imputation(merged_df, 'director', ['listed_in', 'country', 'type'])

# **Basis Cleaning and Exploration**

In [58]:
# Dropping Duplicated Columns
df = df.drop_duplicates(subset=df.columns, keep="first", inplace=False)
df.shape[0]

8807

# **Date_Added**

In [59]:
# Remove leading and trailing spaces, extract only the date part, and convert to datetime format

df["date_added"] = (df["date_added"]
                    .astype(str)
                    .str.strip()
                    .apply(pd.to_datetime)
                    .dt.date
                    #.apply(pd.to_datetime)
                    )
df["date_added"].head(3)

# df["date_added"].describe()

0    2021-09-25
1    2021-09-24
2    2021-09-24
Name: date_added, dtype: object

# **Director**

In [60]:
# Create a copy of the DataFrame to work on
df_dir = df[["show_id", "title", "director"]].copy()
df_dir[:10]

# Specific check for the title "Scooby-Doo on Zombie Island"
df_dir[df_dir["title"] == "Scooby-Doo on Zombie Island"]

,show_id,title,director
7959,s7960,Scooby-Doo on Zombie Island,"Hiroshi Aoyama, Kazumi Fukushima, Jim Stenstrum"


In [61]:
# Function to replace 'nan' with pd.NA and split director names
def process_director(director):
    if isinstance(director, str) and director == 'nan':
        return [pd.NA]
    elif isinstance(director, str):
        return director.split(', ')
    else:
        return director  # If it's already a list or any other type

df_dir['director'] = df_dir['director'].apply(process_director)

exploded_dir_df = df_dir.explode('director').reset_index(drop=True)

# Specific check for the title "Scooby-Doo on Zombie Island"
exploded_dir_df[exploded_dir_df['title'] == "Scooby-Doo on Zombie Island"]

,show_id,title,director
8657,s7960,Scooby-Doo on Zombie Island,Hiroshi Aoyama
8658,s7960,Scooby-Doo on Zombie Island,Kazumi Fukushima
8659,s7960,Scooby-Doo on Zombie Island,Jim Stenstrum


# **Cast**

In [62]:
# Create a copy of the DataFrame to work on
df_cast = df[["show_id", "title", "cast"]].copy()
df_cast[:10]

# Specific check for the title "Blood & Water"
df_cast[df_cast["title"] == "Blood & Water"]


,show_id,title,cast
1,s2,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban..."


In [63]:
# Function to replace 'nan' with pd.NA and split cast names
def process_cast(cast):
    if isinstance(cast, str) and cast == 'nan':
        return [pd.NA]
    elif isinstance(cast, str):
        return cast.split(', ')
    else:
        return cast  # If it's already a list or any other type

df_cast['cast'] = df_cast['cast'].apply(process_cast)
exploded_cast_df = df_cast.explode('cast').reset_index(drop=True)

# Specific check for the title "Blood & Water"
exploded_cast_df[exploded_cast_df['title'] == "Blood & Water"][:3]   #limiting to 3 rows


,show_id,title,cast
1,s2,Blood & Water,Ama Qamata
2,s2,Blood & Water,Khosi Ngema
3,s2,Blood & Water,Gail Mabalane


# **Listed in**

In [64]:
# Create a copy of the DataFrame to work on
df_listed_in = df[["show_id", "title", "listed_in"]].copy()
df_listed_in[:10]

# Specific check for the title "Blood & Water"
df_listed_in[df_listed_in["title"] == "Blood & Water"]

,show_id,title,listed_in
1,s2,Blood & Water,"International TV Shows, TV Dramas, TV Mysteries"


In [65]:
# Function to replace 'nan' with pd.NA and split listed_in entries
def process_listed_in(listed_in):
    if isinstance(listed_in, str) and listed_in == 'nan':
        return [pd.NA]
    elif isinstance(listed_in, str):
        return listed_in.split(', ')
    else:
        return listed_in  # If it's already a list or any other type

df_listed_in['listed_in'] = df_listed_in['listed_in'].apply(process_listed_in)
exploded_listed_in_df = df_listed_in.explode('listed_in').reset_index(drop=True)

# Specific check for the title "Blood & Water"
exploded_listed_in_df[exploded_listed_in_df['title'] == "Blood & Water"]


,show_id,title,listed_in
1,s2,Blood & Water,International TV Shows
2,s2,Blood & Water,TV Dramas
3,s2,Blood & Water,TV Mysteries


# **Country**

In [66]:
# Create a copy of the DataFrame to work on
df_country = df[["show_id", "title", "country"]].copy()
df_country[:10]

# Specific check for the title "Yellowbird"
df_country[df_country["title"] == "Yellowbird"]

,show_id,title,country
8776,s8777,Yellowbird,"France, Belgium"


In [67]:
# Function to replace 'nan' with pd.NA and split country entries
def process_country(country):
    if isinstance(country, str) and country == 'nan':
        return [pd.NA]
    elif isinstance(country, str):
        return country.split(', ')
    else:
        return country  # If it's already a list or any other type

# Assuming df_country is your DataFrame containing the 'country' column
df_country['country'] = df_country['country'].apply(process_country)
exploded_country_df = df_country.explode('country').reset_index(drop=True)

# Specific check for the title "Yellowbird"
exploded_country_df[exploded_country_df['title'] == "Yellowbird"]


,show_id,title,country
10802,s8777,Yellowbird,France
10803,s8777,Yellowbird,Belgium


# **Merging the Unnested DataFrame**

In [68]:
# Chained merge operation
merged_df = (exploded_dir_df
             .merge(exploded_cast_df, on=['show_id', 'title'], how='inner')
             .merge(exploded_listed_in_df, on=['show_id', 'title'], how='inner')
             .merge(exploded_country_df, on=['show_id', 'title'], how='inner')
             .merge(df[['show_id', 'type', 'title', 'date_added', 'release_year', 'rating', 'duration', 'description']], on=['show_id', 'title'], how='inner')
            )

# merged_df[:10]
data_quality_statistics(merged_df).query("Percentage_of_NaN > 0")

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,50643,25.07,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,11897,5.89,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [69]:
# Export Backup
# merged_df.to_pickle('backup1.pkl')

In [101]:
# Import Backup
# merged_df = pd.read_pickle('backup1.pkl')

In [102]:
# Imputing Director Column
imputed_df_1 = mode_based_imputation(merged_df,
                                    'director',
                                    ['listed_in', 'country', 'type'])

data_quality_statistics(imputed_df_1).query("Percentage_of_NaN > 0")  #25.07% to 6.28%

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,12678,6.28,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,11897,5.89,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [103]:
# Imputing Director Column
imputed_df_2a = mode_based_imputation(imputed_df_1,
                                      'director',
                                      ['country', 'type'])

data_quality_statistics(imputed_df_2a).query("Percentage_of_NaN > 0")  #6.28% to 3.59%

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,7256,3.59,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,11897,5.89,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [104]:
# Imputing Director Column
imputed_df_2b = mode_based_imputation(imputed_df_2a,
                                      'director',
                                      ['country'])

data_quality_statistics(imputed_df_2b).query("Percentage_of_NaN > 0")  #3.59% to 2.49%

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,5021,2.49,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,11897,5.89,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [106]:
# Export Backup
# imputed_df_2b.to_pickle('backup2.pkl')
# Import Backup
# imputed_df_2b = pd.read_pickle('backup2.pkl')

In [112]:
# Imputing Country Column
imputed_df_3 = mode_based_imputation(imputed_df_2b,
                                      'country',
                                      ['director'])

data_quality_statistics(imputed_df_3).query("Percentage_of_NaN > 0")  #5.89% to 4.56%

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,5021,2.49,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,9203,4.56,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [114]:
# Export Backup
# imputed_df_3.to_pickle('backup3.pkl')
# Import Backup
# imputed_df_3 = pd.read_pickle('backup3.pkl')

In [115]:
# Imputing Country Column
imputed_df_4 = mode_based_imputation(imputed_df_3,
                                    'country',
                                    ['cast'])

data_quality_statistics(imputed_df_4).query("Percentage_of_NaN > 0")  #4.56% to 2.49%

,Column,Number_of_Rows,Number_of Unique Values,Number_of_NaN,Percentage_of_NaN,Data_Type
2,director,201991,4993,5021,2.49,object
3,cast,201991,36439,2146,1.06,object
5,country,201991,127,5030,2.49,object
7,date_added,201991,1714,158,0.08,object
9,rating,201991,17,67,0.03,object


In [117]:
# Export Backup
# imputed_df_4.to_pickle('backup4.pkl')
# Import Backup
# imputed_df_4 = pd.read_pickle('backup4.pkl')

In [129]:
len(imputed_df_4[imputed_df_4["director"].isna() & imputed_df_4["country"].isna()])/201991*100

# imputed_df_4.query("director.isnull() or cast.isnull() or country.isnull() or date_added.isnull() or rating.isnull()")

1.3931313771405658

In [127]:
len(imputed_df_4.query("director.isnull() or cast.isnull() or country.isnull() or date_added.isnull() or rating.isnull()"))

9324

In [128]:
9324/201991*100

4.6160472496299345

In [ ]:
break

In [52]:
break--

SyntaxError: invalid syntax (<ipython-input-52-6420b162c802>, line 1)

**Mode Imputation on Director Feature**

In [ ]:
# # Drop rows where 'director' is null
# filtered_df = merged_df.dropna(subset=['director'])

# # Define a function to sort directors by frequency
# def sort_directors_by_frequency(directors):
#     if directors.empty:
#         return None, None
#     director_count = directors.value_counts()  # Count occurrences of each director
#     sorted_directors = director_count.index.tolist()  # Sort directors by frequency
#     return ', '.join(sorted_directors), sorted_directors[0]

# # Group by and perform string aggregation with sorted directors
# grouped_df = (filtered_df
#               .groupby(["listed_in", "country", "type"])["director"]
#               .apply(sort_directors_by_frequency)
#               .apply(pd.Series)
#               )

# # Rename columns
# grouped_df.columns = ['sorted_directors', 'first_director']

# # Reset index
# grouped_df = grouped_df.reset_index()
# grouped_df[:10]

In [ ]:
# # Create a dictionary mapping group keys to first director names
# group_first_directors = {}
# for index, row in grouped_df.iterrows():
#     key = (row["listed_in"], row["country"], row["type"])
#     group_first_directors[key] = row["first_director"]

# # Fill missing values in merged_df["director"] based on the mapping
# for index, row in merged_df.iterrows():
#     key = (row["listed_in"], row["country"], row["type"])
#     if pd.isna(row["director"]):
#         merged_df.at[index, "director"] = group_first_directors.get(key, row["director"])

# merged_df[:5]

In [ ]:
# x1 = round(len(merged_df[merged_df["director"].isna()])/len(merged_df)*100,2)
# x2 = round(len(merged_df[merged_df["country"].isna()])/len(merged_df)*100,2)
# y = round(len(merged_df[merged_df["director"].isna() & merged_df["country"].isna()])/len(merged_df)*100,2)
# x1,x2,y

In [ ]:
import

**Mode Imputation on Country Feature**

In [ ]:
# Drop rows where 'country' is null
filtered_df_country = merged_df.dropna(subset=['country'])

# Define a function to sort countries by frequency
def sort_countries_by_frequency(countries):
    if countries.empty:
        return None, None
    country_count = countries.value_counts()  # Count occurrences of each country
    sorted_countries = country_count.index.tolist()  # Sort countries by frequency
    return ', '.join(sorted_countries), sorted_countries[0]

# Group by and perform string aggregation with sorted countries
grouped_df_country = (filtered_df_country
                      .groupby(["listed_in", "type"
                      , "cast"
                                ]
                               )["country"]
                      .apply(sort_countries_by_frequency)
                      .apply(pd.Series)
                      )

# Rename columns
grouped_df_country.columns = ['sorted_countries', 'first_country']

# Reset index
grouped_df_country = grouped_df_country.reset_index()
grouped_df_country[:10]


In [ ]:
# Create a dictionary mapping group keys to first country names
group_first_countries = {}
for index, row in grouped_df_country.iterrows():
    key = (row["listed_in"], row["type"]
           , row["cast"]
           )
    group_first_countries[key] = row["first_country"]

# Fill missing values in merged_df["country"] based on the mapping
for index, row in merged_df.iterrows():
    key = (row["listed_in"], row["type"]
            , row["cast"]
           )
    if pd.isna(row["country"]):
        merged_df.at[index, "country"] = group_first_countries.get(key, row["country"])

merged_df[:5]


In [ ]:
x1 = round(len(merged_df[merged_df["director"].isna()])/len(merged_df)*100,2)
x2 = round(len(merged_df[merged_df["country"].isna()])/len(merged_df)*100,2)
y = round(len(merged_df[merged_df["director"].isna() & merged_df["country"].isna()])/len(merged_df)*100,2)
x1,x2,y

In [ ]:
d = data_quality_statistics(merged_df)
d[d["Percentage of NaN"] > 0].reset_index(drop=True)

In [ ]:
merged_df[merged_df["date_added"].isna()]